<a href="https://colab.research.google.com/github/slowe8/MAT421/blob/main/ModuleCHomework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT 421 Module C Homework 19.1 - 19.5
Name: Sean Lowe<br>
ASURITE: slowe8<br>
ASU ID: 1221120472<br>

**19.1 Root Finding Problem Statement**

Being able to determine the roots, or zeros or solutions, of a function is a very valuable skill. In school, we find roots to basic functions using various mathematical methods, however, for complex functions, this process can be very difficult. Instead of finding these roots by hand, we turn to computers to approximate the roots of arbitrary funcitons for us. Using the same numerical methods we could use by hand, we can write a program that approximates the roots of a program by crawling along the function to determine where the root is. In python, the basic function fsolve from the scipy library is used to find the roots of a function.

In [37]:
import numpy as np
from scipy import optimize

f = lambda x: x**2 - 4
r = optimize.fsolve(f, [-5, 5])
print("Roots of f(x) = x^2 - 2: ", r)

Roots of f(x) = x^2 - 2:  [-2.  2.]


We know that these roots are correct from a simple factoring method.

**19.2 Tolerance**

When using numeric, or iterative, solutions, we need to expect a certain level of error in our answers because the calculation will never fully converge onto the full answer. However, if we are iterating through the process and we get multiple answers that are all within a small error, then we should expect that the last calculation we get is acceptable as the solution. This small window of error is called a tolerance. <br> There is a caviot to using tolerance in a method like this. For example, the function f(x) = x^2 + tol has no real roots, however x = tol/2 is an acceptable solution to the method becuase it is within the tolerance of the algorithm. Situations like this should be avoided because it produces wrong solutions.

**19.3 Bisection Method**

The Bisection Method is based on the Intermediate Value Theorem from calculus. A basic explaination of it is that if there are two points on a smooth, continuous function, then there is guaranteed to be a point between them. By starting with a point above and below zero, the bisection method estimates roots by moving closer to 0 over each iteration.

In [38]:
import numpy as np

def bisection_root(function, bounds, tol):
  '''
    This function takes a function, an array of root boundaries, and a
    tolerance, and return a root for each pair of boundaries in a list
  '''
  roots_found = []

  for bound in bounds:
    if len(bound) != 2:
      raise Exception("Each set of boundaries need to have only two points.")

    a = bound[0]
    b = bound[1]

    #if np.sign(a) == np.sign(b):
    #  raise Exception("Points a and b do not bound a root.")

    midpoint = (a + b) / 2

    if abs(function(midpoint)) < tol:
      return [midpoint]
    elif np.sign(f(a)) == np.sign(f(midpoint)):
        roots_found.append(bisection_root(f, [[midpoint, b]],tol)[0])
    elif np.sign(f(b)) == np.sign(f(midpoint)):
        roots_found.append(bisection_root(f, [[a, midpoint]], tol)[0])

  return roots_found

f = lambda x: x**2 - 4
print("Roots of f(x) = x^2 - 4 with a tolerance of 0.01: ", bisection_root(f, [[-5, 0], [0, 5]], 0.01))
print("Roots of f(x) = x^2 - 4 with a tolerance of 0.0001: ", bisection_root(f, [[-5, 0], [0, 5]], 0.0001))


Roots of f(x) = x^2 - 4 with a tolerance of 0.01:  [-2.001953125, 2.001953125]
Roots of f(x) = x^2 - 4 with a tolerance of 0.0001:  [-2.0000076293945312, 2.0000076293945312]


We can see that the bisection method isn't perfect because it can encounter collisions. In other words, the root boundaries must be close to the root.

In [39]:
try:
  print("Roots of f(x) = x^2 - 4 with a tolerance of 0.01: ", bisection_root(f, [[-100, 100], [-50, 502]], 0.01))
except RecursionError as e:
  print("RecursionError: maximum recursion depth exceeded while calling a Python object")
  print("Given root boundaries are not properly surround a root.")

RecursionError: maximum recursion depth exceeded while calling a Python object
Given root boundaries are not properly surround a root.


Because the boundaries are poorly defined it creates an infinite loop and the method will not be able to find the roots.

**19.4 Newton-Raphson Method**

The Newton-Raphson Method uses the derivative of a function to converge to the answer in a similar manner. Instead of taking the midpoint between two points, it finds the difference between the next guess and previous guess using the tangent line.

In [48]:
def newton_raphson(f, df, x, tol):

  roots_found = []

  if 0 in x:
    print("Divide by 0! You can't start at 0 for this method")
    return []

  for x0 in x:
    if abs(f(x0)) < tol:
      return [x0];
    roots_found.append(newton_raphson(f, df, [x0 - f(x0)/df(x0)], tol)[0])

  return roots_found

f = lambda x: x**3 + 2*x**2 - 0.5
df = lambda x: 3*x**2 + 4*x

roots_nr = newton_raphson(f, df, [-5, -1, 5], 1e-15)
print("Roots of f(x) = x^3 + 2x^2 - 0.5 with very low tolerance (high accuracy): ", roots_nr)

Roots of f(x) = x^3 + 2x^2 - 0.5 with very low tolerance (high accuracy):  [-1.8546376797184618, -0.5969682832373152, 0.4516059629557767]


Because this method does not use two points for each root, there are no collisions, so as long as you have at least one point close to each root, the Newton-Raphson Method will produce all of the real roots of a function.

**19.5 Root Finding in Python**

*fsolve()* from Python's *optimize* library is very similar to the function we wrote for the Newton-Raphson method, however it only takes the function and initial guesses. This method is extremely useful because it can solve for the roots of complex functions that would otherwise be difficult to solve.

In [51]:
roots_op = optimize.fsolve(f, [-5, -1, 5])
print("Roots of f(x) = x^3 + 2x^2 - 0.5: ", roots_op)

Roots of f(x) = x^3 + 2x^2 - 0.5:  [-1.85463768 -0.59696828  0.45160596]


Notice that the roots produced from the SciPy library function is accurate but not to the same precision that can be achieved with the Newton-Raphson method above. In other words, if you are writing an application that requires higher precision, you may want to implement a custom root finding algorithm like the one written above.
